## Generating text for news headlines:
Language Modelling is the core problem for a number of of natural language processing tasks such as speech to text, conversational system, and text summarization. A trained language model learns the likelihood of occurrence of a word/ character based on the previous sequence of words/ characters used in the text. Language models can be operated at character level, n-gram level, sentence level or even paragraph level. We will create a language model for predicting next word by implementing and training state-of-the-art Recurrent Neural Networks under Deep Learning.

We will use **Keras** library in python to implement our project.

### 1. Import the libraries

In [1]:
import pandas as pd
import numpy as np
import string

import warnings
warnings.filterwarnings("ignore")

### 2. Load the dataset

Load the dataset of news headlines

In [2]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def read_file_to_sentences(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    sentences = sent_tokenize(text)
    return sentences
all_headlines = read_file_to_sentences("/kaggle/input/corpus2/game_of_thrones.txt")
len(all_headlines)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


24248

In [3]:
all_headlines[:10]

['A Song of Ice and Fire\n\nA Game of Thrones\n\nPROLOGUE\n\nWe should start back, Gared urged as the woods began to grow dark around them.',
 'The wildlings are dead.',
 'Do the dead frighten you?',
 'Ser Waymar Royce asked with just the hint of a smile.',
 'Gared did not rise to the bait.',
 'He was an old man, past fifty, and he had seen the lordlings come and go.',
 'Dead is dead, he said.',
 'We have no business with the dead.',
 'Are they dead?',
 'Royce asked softly.']

### 3. Dataset preparation

#### 3.1 Dataset cleaning

In dataset preparation step, we will first perform text cleaning of the data which includes removal of punctuations and lower casing all the words.

In [4]:
import string

def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt

corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

['a song of ice and fire\n\na game of thrones\n\nprologue\n\nwe should start back gared urged as the woods began to grow dark around them',
 'the wildlings are dead',
 'do the dead frighten you',
 'ser waymar royce asked with just the hint of a smile',
 'gared did not rise to the bait',
 'he was an old man past fifty and he had seen the lordlings come and go',
 'dead is dead he said',
 'we have no business with the dead',
 'are they dead',
 'royce asked softly']

#### 3.2 Make an id for each word in corpus
The next step is Tokenization. Tokenization is a process of extracting tokens (terms / words) from a corpus. Python’s library Keras has inbuilt model for tokenization which can be used to obtain the tokens and their index in the corpus. After this step, every text document in the dataset is converted into sequence of tokens.


In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
token_list = tokenizer.texts_to_sequences(["I am happy to see you here today"])[0]
print(token_list)

check=[]

for i in range(1, len(token_list)):
  n_gram_sequence = token_list[:i+1]
  check.append(n_gram_sequence)

[15, 187, 2292, 3, 77, 10, 83, 831]


In [6]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    ## convert data to sequence of tokens
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)

inp_sequences[:10], total_words

([[4, 1031],
  [4, 1031, 5],
  [4, 1031, 5, 553],
  [4, 1031, 5, 553, 2],
  [4, 1031, 5, 553, 2, 256],
  [4, 1031, 5, 553, 2, 256, 4],
  [4, 1031, 5, 553, 2, 256, 4, 1293],
  [4, 1031, 5, 553, 2, 256, 4, 1293, 5],
  [4, 1031, 5, 553, 2, 256, 4, 1293, 5, 2085],
  [4, 1031, 5, 553, 2, 256, 4, 1293, 5, 2085, 7573]],
 12116)

In the above output [1119, 1120], [1119, 1120,116], [1119, 1120, 116, 1121] and so on represents the ngram phrases generated from the input data. where every integer corresponds to the index of a particular word in the complete vocabulary of words present in the text. For example

**Headline:** i stand  with the shedevils  
**Ngrams:** | **Sequence of Tokens**

<table>
<tr><td>Ngram </td><td> Sequence of Tokens</td></tr>
<tr> <td>i stand </td><td> [30, 507] </td></tr>
<tr> <td>i stand with </td><td> [30, 507, 11] </td></tr>
<tr> <td>i stand with the </td><td> [30, 507, 11, 1] </td></tr>
<tr> <td>i stand with the shedevils </td><td> [30, 507, 11, 1, 975] </td></tr>
</table>



#### 3.3 Padding the Sequences and obtain Variables : Predictors and Target
Now we have generated a data-set which contains sequence of tokens. Before starting training the model, we need to pad the sequences and make their lengths equal. We can use pad_sequence function of Kears for this purpose. To input this data into a learning model, we need to create predictors and label. For example:


Headline:  they are learning data science

<table>
<tr><td>PREDICTORS </td> <td>           LABEL </td></tr>
<tr><td>they                   </td> <td>  are</td></tr>
<tr><td>they are               </td> <td>  learning</td></tr>
<tr><td>they are learning      </td> <td>  data</td></tr>
<tr><td>they are learning data </td> <td>  science</td></tr>
</table>

####Pad all sentences from start to max length , make labels one hot encoded

if input_sequences = [[0, 0, 1, 2, 3], [0, 2, 3, 4, 5], [0, 0, 0, 3, 4]]
then
predictors will be all elements except last element :
predictors = [[0, 0, 1, 2], [0, 2, 3, 4], [0, 0, 0, 3]]
while labels will be the last element :
label = [3, 5, 4]
then
label = [
  [0, 0, 0, 1, 0, 0],  # for label 3
  [0, 0, 0, 0, 0, 1],  # for label 5
  [0, 0, 0, 0, 1, 0]   # for label 4
]


In [7]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
predictors,label,len(label[0]),max_sequence_len

(array([[   0,    0,    0, ...,    0,    0,    4],
        [   0,    0,    0, ...,    0,    4, 1031],
        [   0,    0,    0, ...,    4, 1031,    5],
        ...,
        [   0,    0,    0, ...,   10,   11,    7],
        [   0,    0,    0, ...,   11,    7, 1382],
        [   0,    0,    0, ...,    7, 1382, 1026]], dtype=int32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 12116,
 131)

Perfect, now we can obtain the input vector X (predictors) and the label vector Y (label) which can be used for the training purposes. Now we will create RNN model for out data.

### 4. RNN for Text Generation
Unlike Feed-forward neural networks in which activation outputs are propagated only in one direction, the activation outputs from neurons propagate in both directions (from inputs to outputs and from outputs to inputs) in Recurrent Neural Networks. This creates loops in the neural network architecture which acts as a ‘memory state’ of the neurons. This state allows the neurons an ability to remember what have been learned so far.

The memory state in RNNs gives an advantage over traditional neural networks. Lets architecture a RNN model in our code. We have added total three layers in the model.

1. Input Layer : Takes the sequence of words as input
2. RNN Layer : Computes the output using RNN units. We have added 200 units in the layer.
3. Dropout Layer : A regularisation layer which randomly turns-off the activations of some neurons in the RNN layer. It helps in preventing over fitting.
4. Output Layer : Computes the probability of the best possible next word as output

We will run this model for total 100 epoochs but it can be experimented further.

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Define the dataset
class TextDataset(Dataset):
    def __init__(self, predictors, labels):
        self.predictors = predictors
        self.labels = labels

    def __len__(self):
        return len(self.predictors)

    def __getitem__(self, idx):
        return self.predictors[idx], self.labels[idx]

# Define the model class
class TextGenerationModel(nn.Module):
    def __init__(self, total_words, embedding_dim, rnn_units):
        super(TextGenerationModel, self).__init__()
        self.embedding = nn.Embedding(total_words, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, rnn_units, batch_first=True)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(rnn_units, total_words)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.rnn(x)
        x = self.dropout(x[:, -1, :])  # Only take the output from the last time step
        x = self.fc(x)
        return x

# Function to train the model
def train_model(model, dataset, epochs, batch_size, learning_rate, patience, device):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=patience, min_lr=1e-5)

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for batch_idx, (data, target) in enumerate(dataloader):
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)

            # Flatten the output and target for the loss function
            output = output.view(-1, output.size(-1))
            target = target.view(-1)
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_epoch_loss = epoch_loss / len(dataloader)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_epoch_loss:.4f}')
        scheduler.step(avg_epoch_loss)

# Parameters
total_words = 10000  # Example value; use the actual value from your data
embedding_dim = 32
rnn_units = 200
max_sequence_len = 50  # Example value; use the actual value from your data
batch_size = 32
learning_rate = 0.001
epochs = 100
patience = 5

# Example data (replace with actual data)
predictors = np.random.randint(0, total_words, (1000, max_sequence_len-1))  # Dummy data
labels = np.random.randint(0, total_words, 1000)  # Dummy data

# Convert data to tensors
predictors = torch.tensor(predictors, dtype=torch.long)
labels = torch.tensor(labels, dtype=torch.long)

# Check tensor shapes and device
print(f"Predictors shape: {predictors.shape}, Labels shape: {labels.shape}")
print(f"Predictors device: {predictors.device}, Labels device: {labels.device}")

# Prepare dataset
dataset = TextDataset(predictors, labels)

# Initialize and train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TextGenerationModel(total_words, embedding_dim, rnn_units).to(device)

# Train the model
train_model(model, dataset, epochs, batch_size, learning_rate, patience, device)


Predictors shape: torch.Size([1000, 49]), Labels shape: torch.Size([1000])
Predictors device: cpu, Labels device: cpu
Epoch 1/100, Loss: 9.2294
Epoch 2/100, Loss: 7.9520
Epoch 3/100, Loss: 7.0736
Epoch 4/100, Loss: 6.5865
Epoch 5/100, Loss: 6.1579
Epoch 6/100, Loss: 5.6549
Epoch 7/100, Loss: 5.0389
Epoch 8/100, Loss: 4.3272
Epoch 9/100, Loss: 3.5775
Epoch 10/100, Loss: 2.8011
Epoch 11/100, Loss: 2.0762
Epoch 12/100, Loss: 1.4474
Epoch 13/100, Loss: 0.9579
Epoch 14/100, Loss: 0.6314
Epoch 15/100, Loss: 0.4311
Epoch 16/100, Loss: 0.3157
Epoch 17/100, Loss: 0.2334
Epoch 18/100, Loss: 0.1845
Epoch 19/100, Loss: 0.1508
Epoch 20/100, Loss: 0.1265
Epoch 21/100, Loss: 0.1079
Epoch 22/100, Loss: 0.0921
Epoch 23/100, Loss: 0.0818
Epoch 24/100, Loss: 0.0743
Epoch 25/100, Loss: 0.0635
Epoch 26/100, Loss: 0.0591
Epoch 27/100, Loss: 0.0538
Epoch 28/100, Loss: 0.0492
Epoch 29/100, Loss: 0.0449
Epoch 30/100, Loss: 0.0409
Epoch 31/100, Loss: 0.0372
Epoch 32/100, Loss: 0.0353
Epoch 33/100, Loss: 0.0325


Lets train our model now

In [ ]:
model.fit(predictors, label, epochs=8)

Epoch 1/8
260626/260626 [==============================] - 532s 2ms/step - loss: 6.5739
Epoch 2/8
205536/260626 [======================>.......] - ETA: 1:53 - loss: 5.9637

## 5. Generating the text

Great, our model architecture is now ready and we can train it using our data. Next lets write the function to predict the next word based on the input words (or seed text). We will first tokenize the seed text, pad the sequences and pass into the trained model to get predicted word. The multiple predicted words can be appended together to get predicted sequence.


In [17]:
import torch
from keras.preprocessing.sequence import pad_sequences
import numpy as np

def generate_text(seed_text, next_words, model, tokenizer, max_sequence_len, device):
    model.eval()  # Set the model to evaluation mode
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        token_list = torch.tensor(token_list, dtype=torch.long).to(device)
        
        with torch.no_grad():
            predicted_probs = model(token_list)
        
        predicted = torch.argmax(predicted_probs, dim=-1).item()  # Get the index of the highest probability

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

# Ensure the model, tokenizer, and device are already defined
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 6. Some Results

In [23]:
print(generate_text("Have ", 5, model, tokenizer, max_sequence_len, device))
print(generate_text("Did", 5, model, tokenizer, max_sequence_len, device))
print(generate_text("We", 6, model, tokenizer, max_sequence_len, device))
print(generate_text(" All the bodies", 3, model, tokenizer, max_sequence_len, device))
print(generate_text("Especially", 4, model, tokenizer, max_sequence_len, device))
print(generate_text("The young knight", 6, model, tokenizer, max_sequence_len, device))

Have  Fiery Bends Sneaking Eighth Favored
Did Magister Mornings Humpbacked Stalking Debt
We Glistened Bends Startledand Shields Hallooed Claws
 All The Bodies Favored Outrange Obey
Especially Tale List Tide Impressive
The Young Knight Partner Fashioned Helplessly Lemon Reward Notched


In [29]:
print(generate_text("Nobody will feel", 4, model, tokenizer, max_sequence_len, device))

Nobody Will Feel Lolling Outside Silverpale Leffords
